# Concordance Analysis: Positive SPECT/CT and 4D CT Scans

### Objective

Evaluate **localization concordance with surgical findings** for patients with **positive SPECT/CT scans** and **positive 4D CT scans**, assessing **each modality’s diagnostic accuracy individually**.

> **Note:** Analyses were performed **separately for each modality**, not as a direct comparison between them.

### Data Source

- `side_concordance_results.xlsx` containing pre-operative imaging results and surgical outcomes.

### Outputs

- Concordance rates (% exact gland or side match) calculated **individually** for:
  - SPECT/CT scans
  - 4D CT scans
- Interpretation notes for manuscript reporting
 ---

In [15]:
import pandas as pd 
preop_imaging_df = pd.read_excel('../../Data/final_imaging_data.xlsx')
import matplotlib.pyplot as plt


In [21]:
def check_concordance(row, modality_col):
    imaging = row[modality_col]
    surgical = row['Surgical Findings']

    # Return Unknown if imaging or surgical data is missing
    if pd.isnull(imaging) or pd.isnull(surgical):
        return 'Unknown'

    # Handle No Scan and Non-localizing 
    if imaging == 'No Scan':
        return 'No Scan'
    if imaging == 'Non-localizing':
        return 'Non-localizing'

    # If surgery found 4 gland hyperplasia, any single imaging finding is incorrect
    if 'hyperplasia' in surgical.lower():
        return 'Incorrect Localization'

    # Split and clean imaging and surgical gland strings into sets
    imaging_glands = [g.strip().lower() for g in imaging.split(',')]
    surgical_glands = [g.strip().lower() for g in surgical.split(',')]
    imaging_set = set(imaging_glands)
    surgical_set = set(surgical_glands)

    # Exact match: all surgical glands found in imaging
    if surgical_set.issubset(imaging_set):
        return 'Yes (Exact)'

    # Partial match: at least one gland match
    if imaging_set & surgical_set:
        return 'Yes (Partial)'

    # Side match analysis
    imaging_sides = ['left' for ig in imaging_glands if 'left' in ig] + \
                    ['right' for ig in imaging_glands if 'right' in ig]
    surgical_sides = ['left' for sg in surgical_glands if 'left' in sg] + \
                     ['right' for sg in surgical_glands if 'right' in sg]
    
    # Multi-gland surgery with side match → Partial
    if any(side in surgical_sides for side in imaging_sides):
        if len(surgical_glands) > 1:
            return 'Yes (Partial)'
        else:
            # Single gland surgery with side match → Side match
            return 'Yes (Side)'
    return 'Incorrect Localization'

In [22]:

# Filter to scans that localized (Left or Right)
localized_4dct = preop_imaging_df[
    preop_imaging_df['4D CT Imaging Side'].isin(['Left', 'Right'])
].copy()

total_localized_4dct = localized_4dct.shape[0]

# Calculate total concordant (either exact or side match) using existing calculated column
total_concordant = localized_4dct['4D CT Scan Calculated Concordance'].isin(['Yes (Exact)', 'Yes (Side)', 'Yes (Partial)']).sum()

# Calculate percentage
side_concordance_percentage_4dct = (total_concordant / total_localized_4dct * 100) if total_localized_4dct > 0 else None

# Create results dataframe
side_concordance_4dct_df = pd.DataFrame({
    'Metric': [
        'Total 4D CT scans with localization',
        'Scans matching surgical side or exact gland'
    ],
    'Value': [
        total_localized_4dct,
        f"{total_concordant} ({side_concordance_percentage_4dct:.2f}%)" if side_concordance_percentage_4dct is not None else 'N/A'
    ]
})

# Display results
display(side_concordance_4dct_df)

,Metric,Value
0,Total 4D CT scans with localization,171
1,Scans matching surgical side or exact gland,146 (85.38%)


In [ ]:
# Define modality and imaging side column
modality = 'SPECT/CT'
imaging_side_col = 'SPECT Imaging Side'

# Filter to scans that localized (Left or Right)
localized_spect = preop_imaging_df[
    preop_imaging_df[imaging_side_col].isin(['Left', 'Right'])
].copy()

total_localized_spect = localized_spect.shape[0]

# Calculate total concordant (Exact, Side, or Partial match) using existing calculated column
total_concordant = localized_spect['SPECT/CT Calculated Concordance'].isin(
    ['Yes (Exact)', 'Yes (Side)', 'Yes (Partial)']
).sum()

# Calculate percentage
side_concordance_percentage_spect = (total_concordant / total_localized_spect * 100) if total_localized_spect > 0 else None

# Create results dataframe
side_concordance_spect_df = pd.DataFrame({
    'Metric': [
        'Total SPECT/CT scans with localization',
        'Scans matching surgical side, partial, or exact gland'
    ],
    'Value': [
        total_localized_spect,
        f"{total_concordant} ({side_concordance_percentage_spect:.2f}%)" if side_concordance_percentage_spect is not None else 'N/A'
    ]
})

# Display results
display(side_concordance_spect_df)

,Metric,Value
0,Total SPECT/CT scans with localization,177
1,"Scans matching surgical side, partial, or exac...",163 (92.09%)


In [24]:
# Combine SPECT/CT and 4D CT side concordance dataframes with a Modality column
combined_side_concordance_df = pd.concat(
    [side_concordance_spect_df.assign(Modality='SPECT/CT'),
     side_concordance_4dct_df.assign(Modality='4D CT Scan')],
    ignore_index=True
)[['Modality', 'Metric', 'Value']]

# Display combined results
display(combined_side_concordance_df)

,Modality,Metric,Value
0,SPECT/CT,Total SPECT/CT scans with localization,177
1,SPECT/CT,"Scans matching surgical side, partial, or exac...",163 (92.09%)
2,4D CT Scan,Total 4D CT scans with localization,171
3,4D CT Scan,Scans matching surgical side or exact gland,146 (85.38%)


---

### Interpretation

- **SPECT/CT:** Demonstrated a concordance rate of **92%**, indicating strong reliability in localizing the abnormal gland to the correct side or exact gland location.

- **4D CT:** Showed a concordance rate of **85%**, supporting its role as a highly useful imaging modality, though slightly lower than SPECT/CT in this cohort.

- **Clinical implication:** Both modalities provide valuable pre-operative localization data. The high concordance of SPECT/CT may reflect institutional expertise or protocol differences.

- **Next steps:** Compare these findings to PPV and sensitivity results to assess overall diagnostic performance and guide modality selection recommendations.